In [1]:
import coremltools
from PIL import Image
import os
import re
import shutil
import json
import yaml

## Open source model

In [2]:
model_folder = '/Users/landyrev/Downloads/latest'

model_name = filter(
    lambda x: re.match(r'.*.caffemodel', x, re.IGNORECASE), 
    os.listdir(model_folder)
)[0]

model_path = os.path.join(model_folder, model_name)

## Load and convert model

In [3]:
m = coremltools.converters.caffe.convert(
    (
        model_path, os.path.join(model_folder, 'deploy.prototxt'), 
        os.path.join(model_folder, 'mean.binaryproto')
    ),
    class_labels=os.path.join(model_folder, 'labels.txt'), 
    image_input_names='data'
)

## Add model metadata

In [4]:
m.author = 'Aleksey Landyrev'
m.short_description = 'How to draw model'
m.save('realmodel.mlmodel')

## Copy model into target project

In [6]:
shutil.copy('./realmodel.mlmodel', '../../htd/Data/realmodel.mlmodel')

## Copy needed images

In [8]:
!ls ../

AllImages converter


In [13]:
tutorials = []

shutil.rmtree('../../htd/Data/Images.bundle')
os.mkdir('../../htd/Data/Images.bundle')

for label in open(os.path.join(model_folder, 'labels.txt')):
    label = label[:-1].replace(" ", "_")
    if label == "random":
        continue
    
    src_dir = os.path.join('../AllImages/', label)
    dst_dir = os.path.join('../../htd/Data/Images.bundle/', label)
    meta_file = os.path.join(src_dir, 'meta.yaml')
    
    os.mkdir(dst_dir)
    
    for image in os.listdir(src_dir):
        if not re.match(r'.*(\.png|\.jpg)', image, re.IGNORECASE):
            continue
        shutil.copy(
            os.path.join(src_dir, image),
            os.path.join(dst_dir, label + "_" + image)
        )
    
    tutorial = {
        "name": label,
        "preview": os.path.join(label, label+ '_preview.png'),
        "tutorials": sorted([os.path.join(label, x) 
             for x in os.listdir(dst_dir)
             if x.endswith('.jpg')
            ])
    }
    print tutorial
    if os.path.isfile(meta_file):
        tutorial.update(yaml.load(open(meta_file)))
    tutorials.append(tutorial)

yaml.dump(
    tutorials,
    open("../../htd/Data/Images.bundle/levels.yaml", "w")
)

{'tutorials': ['lutz_aquarium0/lutz_aquarium0_00.jpg', 'lutz_aquarium0/lutz_aquarium0_01.jpg', 'lutz_aquarium0/lutz_aquarium0_02.jpg', 'lutz_aquarium0/lutz_aquarium0_03.jpg', 'lutz_aquarium0/lutz_aquarium0_04.jpg'], 'preview': 'lutz_aquarium0/lutz_aquarium0_preview.png', 'name': 'lutz_aquarium0'}
{'tutorials': ['lutz_boat0/lutz_boat0_00.jpg', 'lutz_boat0/lutz_boat0_01.jpg', 'lutz_boat0/lutz_boat0_02.jpg', 'lutz_boat0/lutz_boat0_03.jpg', 'lutz_boat0/lutz_boat0_04.jpg', 'lutz_boat0/lutz_boat0_05.jpg'], 'preview': 'lutz_boat0/lutz_boat0_preview.png', 'name': 'lutz_boat0'}
{'tutorials': ['lutz_boat1/lutz_boat1_00.jpg', 'lutz_boat1/lutz_boat1_01.jpg', 'lutz_boat1/lutz_boat1_02.jpg', 'lutz_boat1/lutz_boat1_03.jpg', 'lutz_boat1/lutz_boat1_04.jpg', 'lutz_boat1/lutz_boat1_05.jpg'], 'preview': 'lutz_boat1/lutz_boat1_preview.png', 'name': 'lutz_boat1'}
{'tutorials': ['lutz_buffalo0/lutz_buffalo0_00.jpg', 'lutz_buffalo0/lutz_buffalo0_01.jpg', 'lutz_buffalo0/lutz_buffalo0_02.jpg', 'lutz_buffalo0/lu

{'tutorials': ['lutz_snail0/lutz_snail0_00.jpg', 'lutz_snail0/lutz_snail0_01.jpg', 'lutz_snail0/lutz_snail0_02.jpg', 'lutz_snail0/lutz_snail0_03.jpg'], 'preview': 'lutz_snail0/lutz_snail0_preview.png', 'name': 'lutz_snail0'}
{'tutorials': ['lutz_squirrel0/lutz_squirrel0_00.jpg', 'lutz_squirrel0/lutz_squirrel0_01.jpg', 'lutz_squirrel0/lutz_squirrel0_02.jpg'], 'preview': 'lutz_squirrel0/lutz_squirrel0_preview.png', 'name': 'lutz_squirrel0'}
{'tutorials': ['lutz_stork0/lutz_stork0_00.jpg', 'lutz_stork0/lutz_stork0_01.jpg', 'lutz_stork0/lutz_stork0_02.jpg', 'lutz_stork0/lutz_stork0_03.jpg', 'lutz_stork0/lutz_stork0_04.jpg', 'lutz_stork0/lutz_stork0_05.jpg'], 'preview': 'lutz_stork0/lutz_stork0_preview.png', 'name': 'lutz_stork0'}
{'tutorials': ['lutz_swallow0/lutz_swallow0_00.jpg', 'lutz_swallow0/lutz_swallow0_01.jpg', 'lutz_swallow0/lutz_swallow0_02.jpg', 'lutz_swallow0/lutz_swallow0_03.jpg', 'lutz_swallow0/lutz_swallow0_04.jpg'], 'preview': 'lutz_swallow0/lutz_swallow0_preview.png', 'nam

## Try to predict test image

In [13]:
def predict(image_name):
    image = Image.open(image_name).convert('L')
    image = image.resize((224, 224))
    features = m.predict({
        'data': image
    }).get('softmax')
    for feature in features:
        print feature, int(features[feature] * 100)
        
predict('/Users/landyrev/Downloads/fvsdfvsdfv.jpg')

lutz_swan0 0
lutz_lighthouse0 0
random 0
lutz_boat0 99
lutz_pug0 0
lutz_woman0 0
